In [1]:
import requests

In [2]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"
data = requests.get(standings_url)

In [3]:
from bs4 import BeautifulSoup

In [24]:
soup = BeautifulSoup(data.text, 'html.parser')
standings_table_list = soup.select('table.stats_table')

if len(standings_table_list) > 0:
    standings_table = standings_table_list[0]
    links = standings_table.find_all('a')
    links = [l.get("href") for l in links]
    links = [l for l in links if '/squads/' in l]
else:
    print("No standings table found.")

In [5]:
team_urls = [f"https://fbref.com{l}" for l in links]

In [17]:
team_urls

['https://fbref.com/en/squads/b8fd03ef/Manchester-City-Stats',
 'https://fbref.com/en/squads/18bb7c10/Arsenal-Stats',
 'https://fbref.com/en/squads/19538871/Manchester-United-Stats',
 'https://fbref.com/en/squads/b2b47a98/Newcastle-United-Stats',
 'https://fbref.com/en/squads/822bd0ba/Liverpool-Stats',
 'https://fbref.com/en/squads/d07537b9/Brighton-and-Hove-Albion-Stats',
 'https://fbref.com/en/squads/8602292d/Aston-Villa-Stats',
 'https://fbref.com/en/squads/361ca564/Tottenham-Hotspur-Stats',
 'https://fbref.com/en/squads/cd051869/Brentford-Stats',
 'https://fbref.com/en/squads/fd962109/Fulham-Stats',
 'https://fbref.com/en/squads/47c64c55/Crystal-Palace-Stats',
 'https://fbref.com/en/squads/cff3d9bb/Chelsea-Stats',
 'https://fbref.com/en/squads/8cec06e1/Wolverhampton-Wanderers-Stats',
 'https://fbref.com/en/squads/7c21e445/West-Ham-United-Stats',
 'https://fbref.com/en/squads/4ba7cbea/Bournemouth-Stats',
 'https://fbref.com/en/squads/e4a775cb/Nottingham-Forest-Stats',
 'https://fbre

In [18]:
data = requests.get(team_urls[0])
data

<Response [200]>

In [19]:
import pandas as pd
matches = pd.read_html(data.text, match="Scores & Fixtures")[0]

In [20]:
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/shooting/' in l]

In [21]:
data = requests.get(f"https://fbref.com{links[0]}")

In [22]:
links[0]

'/en/squads/b8fd03ef/2022-2023/matchlogs/all_comps/shooting/Manchester-City-Match-Logs-All-Competitions'

In [10]:
shooting = pd.read_html(data.text, match="Shooting")[0]

In [11]:
shooting.head()

For Manchester City                                                     \
                 Date   Time              Comp                Round  Day   
0          2022-07-30  17:00  Community Shield  FA Community Shield  Sat   
1          2022-08-07  16:30    Premier League          Matchweek 1  Sun   
2          2022-08-13  15:00    Premier League          Matchweek 2  Sat   
3          2022-08-21  16:30    Premier League          Matchweek 3  Sun   
4          2022-08-27  15:00    Premier League          Matchweek 4  Sat   

                                         ... Standard               Expected  \
     Venue Result GF GA        Opponent  ...     Dist   FK PK PKatt       xG   
0  Neutral      L  1  3       Liverpool  ...      NaN  NaN  0     0      NaN   
1     Away      W  2  0        West Ham  ...     18.7  1.0  1     1      2.2   
2     Home      W  4  0     Bournemouth  ...     17.5  0.0  0     0      1.7   
3     Away      D  3  3   Newcastle Utd  ...     16.2  1.0  0     0      2.1   
4     Home      W  4  2  Crystal Palace  ...     14.1  0.0  0     0      2.2   

                            Unnamed: 25_level_0  
  npxG npxG/Sh G-xG np:G-xG        Match Report  
0  NaN     NaN  NaN     NaN        Match Report  
1  1.4    0.11 -0.2    -0.4        Match Report  
2  1.7    0.09  1.3     1.3        Match Report  
3  2.1    0.10  0.9     0.9        Match Report  
4  2.2    0.13  1.8     1.8        Match Report  

[5 rows x 26 columns]

In [12]:
shooting.columns = shooting.columns.droplevel()

In [13]:
team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")

In [14]:
team_data.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Formation,Referee,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt
0,2022-07-30,17:00,Community Shield,FA Community Shield,Sat,Neutral,L,1.0,3.0,Liverpool,...,4-3-3,Craig Pawson,Match Report,NaN,14,8,NaN,NaN,0,0
1,2022-08-07,16:30,Premier League,Matchweek 1,Sun,Away,W,2.0,0.0,West Ham,...,4-3-3,Michael Oliver,Match Report,NaN,13,1,18.7,1.0,1,1
2,2022-08-13,15:00,Premier League,Matchweek 2,Sat,Home,W,4.0,0.0,Bournemouth,...,4-2-3-1,David Coote,Match Report,NaN,19,7,17.5,0.0,0,0
3,2022-08-21,16:30,Premier League,Matchweek 3,Sun,Away,D,3.0,3.0,Newcastle Utd,...,4-3-3,Jarred Gillett,Match Report,NaN,21,10,16.2,1.0,0,0
4,2022-08-27,15:00,Premier League,Matchweek 4,Sat,Home,W,4.0,2.0,Crystal Palace,...,4-2-3-1,Darren England,Match Report,NaN,18,5,14.1,0.0,0,0


In [15]:
years = list(range(2022, 2002, -1))
all_matches = []

In [16]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [23]:
import time
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "Premier League"]
        
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(1)

KeyError: "['FK'] not in index"

In [ ]:
len(all_matches)

In [ ]:
match_df = pd.concat(all_matches)

In [ ]:
match_df.columns = [c.lower() for c in match_df.columns]

In [ ]:
match_df

In [ ]:
match_df.to_csv("matches_20yr.csv")